In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.2


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=10))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.models.DRMM.get_default_preprocessor()

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7490.71it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4108.48it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 2216408.14it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8109.97it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4097.99it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 501783.51it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 7991.15it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [6]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7fc345373cf8>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fc3b8602f28>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
histgram_callback = mz.dataloader.callbacks.Histogram(
    embedding_matrix, bin_size=30, hist_mode='LCH'
)

trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=10,
    batch_size=20,
    resample=True,
    callbacks=[histgram_callback]
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    batch_size=20,
    callbacks=[histgram_callback]
)

In [9]:
padding_callback = mz.models.DRMM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [10]:
model = mz.models.DRMM()

model.params['task'] = ranking_task
model.params['mask_value'] = 0
model.params['embedding'] = embedding_matrix
model.params['hist_bin_size'] = 30
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DRMM(
  (embedding): Embedding(30058, 300, padding_idx=0)
  (attention): Attention(
    (linear): Linear(in_features=300, out_features=1, bias=False)
  )
  (mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=30, out_features=10, bias=True)
      (1): Tanh()
    )
    (1): Sequential(
      (0): Linear(in_features=10, out_features=1, bias=True)
      (1): Tanh()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)
Trainable params:  9018023


In [11]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [12]:
trainer.run()

[Iter-255 Loss-2.327]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3628 - normalized_discounted_cumulative_gain@5(0.0): 0.4386 - mean_average_precision(0.0): 0.3965



[Iter-510 Loss-2.226]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.349 - normalized_discounted_cumulative_gain@5(0.0): 0.4307 - mean_average_precision(0.0): 0.3899



[Iter-765 Loss-2.202]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3454 - normalized_discounted_cumulative_gain@5(0.0): 0.4241 - mean_average_precision(0.0): 0.3964



[Iter-1020 Loss-2.164]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3677 - normalized_discounted_cumulative_gain@5(0.0): 0.4443 - mean_average_precision(0.0): 0.4112



[Iter-1275 Loss-2.133]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3481 - normalized_discounted_cumulative_gain@5(0.0): 0.4202 - mean_average_precision(0.0): 0.387



[Iter-1530 Loss-2.111]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3653 - normalized_discounted_cumulative_gain@5(0.0): 0.4401 - mean_average_precision(0.0): 0.3986



[Iter-1785 Loss-2.074]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3759 - normalized_discounted_cumulative_gain@5(0.0): 0.4379 - mean_average_precision(0.0): 0.408



[Iter-2040 Loss-2.057]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3229 - normalized_discounted_cumulative_gain@5(0.0): 0.4108 - mean_average_precision(0.0): 0.3733



[Iter-2295 Loss-2.048]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3512 - normalized_discounted_cumulative_gain@5(0.0): 0.4345 - mean_average_precision(0.0): 0.395



[Iter-2550 Loss-2.032]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3577 - normalized_discounted_cumulative_gain@5(0.0): 0.4367 - mean_average_precision(0.0): 0.3932

Cost time: 1020.2365748882294s
